# Creating an NSE table

We can tabulate the NSE state on a grid $(\rho, T, Y_e)$, giving:

* $\bar{A}$ : the mean molecular weight
* $\langle B/A \rangle$ : the average binding energy per nucleon of the NSE state [MeV]
* $dY_e/dt$ : the time-evolution of $Y_e$ due to weak reactions [1/s]
* $d\bar{A}/dt$ : the time-evolution of $\bar{A}$ due to weak reactions [1/s]
* $d\langle B/A\rangle/dt$ : the time-evolution of the binding energy per nucleon due to weak reactions [MeV/s]
* $\epsilon_\nu$ : the energy loss due to neutrinos [erg / g / s] 

In [3]:
import pynucastro as pyna
from pynucastro import Nucleus

First we'll create a simple NSE network with a few nuclei.  We'll include tabulated weak rates so we can compute the change in the state due to electron/positron captures and decays.

In [4]:
nucs = [Nucleus("p"), Nucleus("n"), Nucleus("he4"),
        Nucleus("fe52"), Nucleus("fe53"), Nucleus("fe54"),
        Nucleus("fe55"), Nucleus("fe56"),
        Nucleus("co54"), Nucleus("co55"), Nucleus("co56"),
        Nucleus("ni56"), Nucleus("ni57")]

In [6]:
tl = pyna.TabularLibrary().linking_nuclei(nucs)
rl = pyna.ReacLibLibrary().linking_nuclei(nucs)
all_lib = tl + rl

In [8]:
dupes = all_lib.find_duplicate_links()
rates_to_remove = []
for d in dupes:
    rates_to_remove += [r for r in d if isinstance(r, pyna.rates.ReacLibRate)]

for r in rates_to_remove:
    all_lib.remove_rate(r)

In [9]:
nse = pyna.NSENetwork(libraries=[all_lib])

Now we'll create a grid of temperature, density, and $Y_e$ where we want to compute the NSE state

In [15]:
import numpy as np

In [ ]:
Ts = np.logspace(9.6, 10.4, 3)
rhos = np.logspace(7, 10, 4)
yes = np.linspace(0.43, 0.5, 3)

Finally we can generate the table.  This will compute the NSE state at each combination of $(\rho, T, Y_e)$.  To help accelerate the convergence,
it will start at the highest temperature and loop over $\rho$ and $Y_e$ and cache the values of the proton and neutron chemical potentials for the
next temperature.

In [ ]:
nse.generate_table(rho_values=rhos,
                   T_values=Ts,
                   Ye_values=yes)

The table is stored as `nse.tbl`

In [14]:
%cat nse.tbl

# NSE table generated by pynucastro 2.0.1.post2+g184debe8
# original NSENetwork had 13 nuclei
#
#   log10(rho)       log10(T)           Ye             Abar            <B/A>          dYe/dt         dAbar/dt        d<B/A>/dt         e_nu       
   7.0000000000    9.6000000000    0.5000000000   50.2057480859    8.6280577146  -4.3803693e-05   1.7080366e-17   2.3287883e-06   7.4773253e+13 
   7.0000000000    9.6000000000    0.4650000000   55.7461136880    8.7870609269  -5.5255921e-09  -1.8576389e-21   3.9319171e-10     1.39808e+10 
   7.0000000000    9.6000000000    0.4300000000   11.0637644849    8.1412160175   0.00017699678  -2.9553491e-26  -1.8453762e-12   2.7707299e+14 
   7.0000000000   10.0000000000    0.5000000000    1.2148718034    1.6681978229     0.084056626  -4.1382279e-56   5.7341747e-41   7.8635859e+17 
   7.0000000000   10.0000000000    0.4650000000    1.2129151675    1.6556736890      0.10066497  -2.9530597e-56    3.521478e-41   8.3544486e+17 
   7.0000000000   10.0000000000 

There are other options to the NSE table writing, including storing the mass fractions or a reduced composition constructed by binning the nuclei
into the closest nuclei in a smaller set.